In [31]:
import pandas as pd
import rdflib

g = rdflib.ConjunctiveGraph()
g.parse("mhdb/mhdb.ttl", format='ttl')
g.parse(location=
        ("http://data.bioontology.org/ontologies/MESH/download?apikey=cf2fc1a7-fd1f-48bd-aa9e-e56335ba7235&"
        "format=xml"), format='ttl')

<Graph identifier=http://data.bioontology.org/ontologies/MESH/download?apikey=cf2fc1a7-fd1f-48bd-aa9e-e56335ba7235&format=xml (<class 'rdflib.graph.Graph'>)>

In [33]:
type(g)

rdflib.graph.ConjunctiveGraph

In [ ]:
for s, p, o in g:
    print(" ".join([s, p, o, "\n"]))

In [73]:
qres ="""# "What are all of the top-level mental disorders?"
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
SELECT ?option ?subclass 
WHERE {
    { SELECT ?label ?pl ?subclass
    WHERE {
      ?subclass rdfs:subClassOf* ?intermediate .
      ?intermediate rdfs:subClassOf* :MentalDisorder .
      ?subclass rdfs:label ?label .
      OPTIONAL { ?label skos:prefLabel ?pl }
    }
    GROUP BY ?label ?subclass
    HAVING(count(?intermediate) = 2)
    }
    BIND ( IF ( !bound(?label), ?subclass, IF ( bound(?label), IF ( !bound(?pl), ?label, ?pl), '' ) ) AS ?option )
    }
GROUP BY ?subclass
ORDER BY ?option"""
column_names = ['label', 'iri']
sparql_result = g.query(qres)
options = []
for row in sparql_result:
    options.append([str(row[0]), row[1]])
df = pd.DataFrame(options, columns=column_names)
df.sort_values(by=['label'], axis=0, inplace=True)
print(df)

for key, value in df.iterrows():
    print([type(value['label']), value['iri']])

                                                label  \
0                                    Anxiety Disorder   
1                         Bipolar or Related Disorder   
2                                 Depressive Disorder   
3    Disruptive, Impulse-Control, or Conduct Disorder   
4                               Dissociative Disorder   
19                              Elimination Disorders   
20                       Feeding and Eating Disorders   
5                                    Gender Dysphoria   
6   Medication-Induced Movement Disorder or Other ...   
7                             Neurocognitive Disorder   
8            Obsessive-Compulsive or Related Disorder   
9   Other Conditions That May Be a Focus of Clinic...   
10                              Other Mental Disorder   
11                                Paraphilic Disorder   
12                               Personality Disorder   
13  Schizophrenia Spectrum and Other Psychotic Dis...   
14                             

In [74]:
p = [(1, 2), 1]

In [136]:
column_names = [('label', 'str'), 'iri']
cnames = [c[0] if hasattr(c, '__iter__') else c for c in column_names]
print(cnames)
options = []
for row in sparql_result:
    options.append(["".join([column_names[i][1],"('",r,"')"]) if hasattr(column_names[i],
                    '__iter__') else r for i, r in enumerate(row)])
print(options)

['label', 'i']
[["str('Anxiety Disorder')", "r('http://www.purl.org/mentalhealth#Anxiety_Disorder')"], ["str('Bipolar or Related Disorder')", "r('http://www.purl.org/mentalhealth#Bipolar_or_Related_Disorder')"], ["str('Depressive Disorder')", "r('http://www.purl.org/mentalhealth#Depressive_Disorder')"], ["str('Disruptive, Impulse-Control, or Conduct Disorder')", "r('http://www.purl.org/mentalhealth#Disruptive_Impulse-Control_or_Conduct_Disorder')"], ["str('Dissociative Disorder')", "r('http://www.purl.org/mentalhealth#Dissociative_Disorder')"], ["str('Gender Dysphoria')", "r('http://www.purl.org/mentalhealth#Gender_Dysphoria')"], ["str('Medication-Induced Movement Disorder or Other Adverse Effects of Medication')", "r('http://www.purl.org/mentalhealth#Medication-Induced_Movement_Disorder_or_Other_Adverse_Effects_of_Medication')"], ["str('Neurocognitive Disorder')", "r('http://www.purl.org/mentalhealth#Neurocognitive_Disorder')"], ["str('Obsessive-Compulsive or Related Disorder')", "r('

In [128]:
eval('str(1)')

'1'

In [64]:
qres2 = """PREFIX : <http://www.purl.org/mentalhealth#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
SELECT ?subject ?relationship ?object 
WHERE {
       { :Feeding_or_Eating_Disorder ?relationship ?object }
       UNION
       { ?subject ?relationship :Feeding_or_Eating_Disorder }
       BIND ( IF ( bound(?subject), ?subject, :Feeding_or_Eating_Disorder ) AS ?subject )
       BIND ( IF ( bound(?object), ?object, :Feeding_or_Eating_Disorder ) AS ?object )
}
        """
sparql_result = g.query(qres2)
options = []
for row in sparql_result:
    options.append([[row[0]], [row[1]], [row[2]]])
df = pd.DataFrame(options, columns=['subject', 'relationship', 'object'])
print(df)

                                              subject  \
0   [http://www.purl.org/mentalhealth#Feeding_or_E...   
1   [http://www.purl.org/mentalhealth#Feeding_or_E...   
2   [http://www.purl.org/mentalhealth#Feeding_or_E...   
3   [http://www.purl.org/mentalhealth#Feeding_or_E...   
4   [http://www.purl.org/mentalhealth#Feeding_or_E...   
5   [http://www.purl.org/mentalhealth#Other_Specif...   
6   [http://www.purl.org/mentalhealth#AvoidantRest...   
7   [http://www.purl.org/mentalhealth#Binge-Eating...   
8   [http://www.purl.org/mentalhealth#Anorexia_Ner...   
9   [http://www.purl.org/mentalhealth#Rumination_D...   
10            [http://www.purl.org/mentalhealth#Pica]   
11  [http://www.purl.org/mentalhealth#Bulimia_Nerv...   
12  [http://www.purl.org/mentalhealth#Unspecified_...   

                                         relationship  \
0   [http://www.w3.org/1999/02/22-rdf-syntax-ns#type]   
1        [http://www.w3.org/2000/01/rdf-schema#label]   
2        [http://www.w3.org/20